In [0]:
#exploring better CNN models (ensemble CNN of many neural networks ) https://www.kaggle.com/ankurshukla03/mnist-using-ensemble-cnn-0-997

from google.colab import files
uploaded = files.upload()

In [0]:
import numpy as np


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

In [0]:
import io
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
filetoRead = 'mnist_train.csv'
training_df = pd.read_csv(io.StringIO(uploaded['mnist_train.csv'].decode('utf-8')))

datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [0]:
training_df

In [0]:
y_train1 = pd.DataFrame()
y_train1 = training_df['Category']
X_train1 = training_df.drop(['Id', 'Category'],axis=1)
X_train1

In [0]:
y_train1

In [0]:
y_train = np.array(y_train1.values.tolist(),dtype=int)
X_train = np.array(X_train1.values.tolist(),dtype=int)
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
y_train

array([5, 0, 4, ..., 5, 6, 8])

In [0]:
testing_df = pd.read_csv(io.StringIO(uploaded['mnist_test.csv'].decode('utf-8')))
X_test1 = testing_df.drop(['Id'],axis=1)

In [0]:
testing_df.shape



testing_dataframe


In [0]:
testing_df

In [0]:
d = pd.DataFrame(np.zeros((10000, 1)),dtype=int)
y_test1 = d
y_test1

In [0]:
y_test1.shape

In [0]:
#Threshold
y_test = np.array(y_test1.values.tolist(),dtype=int)
X_test = np.array(X_test1.values.tolist(),dtype=int)

for i in range(len(y_test)):
  for j in range(len(y_test[i])):
    if y_test[i][j] >= 100:
      y_test[i][j] = 1
    else:
      y_test[i][j] = 0
  
for i in range(len(X_test)):
  for j in range(len(X_test[i])):
    if X_test[i][j] >= 100:
      X_test[i][j] = 1
    else:
      X_test[i][j] = 0

In [0]:
X_train = X_train.reshape(X_train.shape[0],28, 28,1)
X_test = X_test.reshape(X_test.shape[0],28, 28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [0]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)


In [0]:
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import LearningRateScheduler
from sklearn.cross_validation import train_test_split

In [0]:
#Run with ensemble average 
nets = 20
model = [0] *nets
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))
    

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# TRAIN NETWORKS
history = [0] * nets
epochs = 45

for j in range(nets):
  X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.1)
  history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
 


In [0]:

results = np.zeros( (X_test.shape[0],1000) ) 
for j in range(nets):
    results = results + model[j].predict(X_test)
results = np.argmax(results,axis = 1)


In [0]:
y_test = results
y_test

In [0]:
#import tensorflow as tf
#model.save('mnist_model.model')
#new_model=tf.keras.models.load_model('mnist_model.model')
#predictions=new_model.predict(X_test)
#pred = []
#for i in range(10000):
#  pred.append(np.argmax(predictions[i]))

#pred  

In [0]:
#np.shape(pred)

In [0]:
#y_test=pred


In [0]:
#y_test
#y_test_eval = np.array(y_test,dtype=int)
#y_test_eval.shape
#y_test_eval = np_utils.to_categorical(y_test_eval, 10)
#y_test_eval = y_test_eval[:,np.newaxis()]
#y_test_eval

#score = model.evaluate(X_test, y_test_eval, verbose=0)
#score

In [0]:
sample_df = pd.read_csv(io.StringIO(uploaded['mnist_sample.csv'].decode('utf-8')))

In [0]:
sample_df['Category'] = y_test
sample_df


In [0]:

sample_df.to_csv("ranjith_mnist.csv",index=False)


In [0]:
files.download("ranjith_mnist.csv")